In [1]:
#import required libraries
import pandas as pd

import os

import csv

from datetime import datetime

import sklearn
from sklearn.cluster import KMeans

import altair as alt
import numpy as np
from sklearn import set_config
from sklearn.model_selection import GridSearchCV, cross_validate, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

import joblib

#create functions and maps for use in program
def time_to_seconds(time_str):
    h, m, s = map(int, time_str.split(':'))
    total_seconds = h * 3600 + m * 60 + s
    return total_seconds

day_to_number = {
    'Mon': 1,
    'Tue': 2,
    'Wed': 3,
    'Thu': 4,
    'Fri': 5,
    'Sat': 6,
    'Sun': 7
}

month_to_number = {
    'Jan': 1,
    'Feb': 2,
    'Mar': 3,
    'Apr': 4,
    'May': 5,
    'Jun': 6,
    'Jul': 7,
    'Aug': 8,
    'Sep': 9,
    'Oct': 10,
    'Nov': 11,
    'Dec': 12
}

#show all columns when displaying data
pd.set_option('display.max_columns', None)

#import the data of the last recorded full day of data
prev_day_data = pd.read_csv('dailydata/Sec_9-5-24.csv')
prev_day_data.tail()

,year,month,num,day,time,price,change_5_ago,change_15_ago,change_30_ago,change_60_ago,change_120_ago,noise_5,noise_15,noise_30,noise_60,noise_120,BBUpper,BBLower,BBdiff,BBUpdiff,BBLowdiff,MFI,MFIchange_5,MFIchange_15,MFIchange_30,MFIchange_60,ATR,ATRchange_5,ATRchange_15,ATRchange_30,ATRchange_60,TTM,TTMchange_5,TTMchange_15,TTMchange_30,color,Mom,Momchange_5,Momchange_15,Momchange_30,Momchange_60,WaveASlow,WaveAFast,WaveBSlow,WaveBFast,WaveCMCAD,WaveCFast,WaveASlowchange_5,WaveASlowchange_15,WaveASlowchange_30,WaveASlowchange_60,WaveAFastchange_5,WaveAFastchange_15,WaveAFastchange_30,WaveAFastchange_60,WaveBSlowchange_5,WaveBSlowchange_15,WaveBSlowchange_30,WaveBSlowchange_60,WaveBFastchange_5,WaveBFastchange_15,WaveBFastchange_30,WaveBFastchange_60,WaveCMCADchange_5,WaveCMCADchange_15,WaveCMCADchange_30,WaveCMCADchange_60,WaveCFastchange_5,WaveCFastchange_15,WaveCFastchange_30,WaveCFastchange_60,VIX,VIXchange_5,VIXchange_15,VIXchange_30,VIXchange_60,VIXchange_120,VIXBBUpper,VIXBBLower,VIXBBdiff,VIXBBUpdiff,VIXBBLowdiff,ADD,TRIN,TRINchange_5,TRINchange_15,TRINchange_30,TRINchange_60,TRIN_lastclose,TICK,TICKBBUpdiff,TICKBBLowdiff,TICKchange_5,TICKchange_15,TICKchange_30,TICKchange_60,TICKBBUpper,TICKBBLower,TICKMA,TICKMADiff,TICKMAchange_5,TICKMAchange_15,TICKMAchange_30,TICKMAchange_60,PCC,PCCchange_5,PCCchange_15,PCCchange_30,PCCchange_60,VOLD,VOLDchange_5,VOLDchange_15,VOLDchange_30,VOLDchange_60,change_5
17780,24.0,9.0,5.0,4.0,53995.0,549.41,0.70,0.80,0.04,-0.80,-1.22,0.158,0.232,0.584,0.588,0.762,551.04,548.37,2.67,-1.63,-1.04,45.07,27.42,13.31,12.47,-8.20,0.738,0.017,0.055,0.126,0.149,-1.03,0.06,-0.197,-0.460,0.0,-0.45,0.89,1.55,0.56,-0.160,-0.061,-0.282,0.614,0.301,-4.64,0.249,0.070,-0.091,-0.202,-1.071,0.083,-0.017,-0.069,-0.815,0.071,-0.119,-0.222,-1.036,0.066,-0.122,-0.244,-1.109,0.08,-0.10,-0.15,-0.86,0.082,-0.093,-0.160,-0.891,20.04,-0.18,-0.2,0.13,0.3,0.03,20.32,19.55,0.77,-0.28,-0.49,-336.0,0.60,0.07,0.05,0.03,0.04,0.74,332.0,-364.293,-1062.493,797.0,23.0,9.0,77.0,696.293,-730.493,62.889,269.111,133.889,522.889,459.889,-538.111,0.85,0.003,0.015,0.074,0.144,-92656917.0,-6356854.0,-27877719.0,-24779388.0,-59340207.0,0.042
17781,24.0,9.0,5.0,4.0,53996.0,549.44,0.75,0.85,0.07,-0.77,-1.19,0.162,0.233,0.584,0.588,0.762,551.04,548.37,2.67,-1.60,-1.07,45.30,27.66,13.54,12.70,-7.96,0.740,0.019,0.057,0.128,0.151,-1.03,0.06,-0.197,-0.460,0.0,-0.42,0.94,1.58,0.59,-0.130,-0.056,-0.277,0.620,0.307,-4.63,0.255,0.079,-0.086,-0.196,-1.066,0.090,-0.012,-0.065,-0.811,0.081,-0.113,-0.216,-1.030,0.076,-0.117,-0.238,-1.103,0.09,-0.09,-0.14,-0.85,0.092,-0.083,-0.154,-0.885,20.04,-0.18,-0.2,0.13,0.3,0.03,20.32,19.55,0.77,-0.28,-0.49,-336.0,0.60,0.07,0.05,0.03,0.04,0.74,332.0,-364.293,-1062.493,797.0,23.0,9.0,77.0,696.293,-730.493,62.889,269.111,69.889,442.889,461.889,-530.111,0.85,0.003,0.015,0.074,0.144,-92656917.0,-6593650.0,-28671473.0,-24771051.0,-59334833.0,0.078
17782,24.0,9.0,5.0,4.0,53997.0,549.48,0.78,0.91,0.11,-0.73,-1.15,0.166,0.235,0.584,0.587,0.762,551.04,548.37,2.67,-1.56,-1.11,45.33,27.69,13.58,12.73,-7.91,0.743,0.022,0.060,0.131,0.154,-1.02,0.07,-0.180,-0.450,0.0,-0.38,0.97,1.66,0.63,-0.105,-0.049,-0.271,0.628,0.315,-4.62,0.264,0.084,-0.072,-0.189,-1.059,0.095,0.000,-0.058,-0.807,0.087,-0.097,-0.208,-1.022,0.082,-0.101,-0.230,-1.105,0.10,-0.07,-0.13,-0.84,0.099,-0.070,-0.146,-0.876,20.04,-0.18,-0.2,0.13,0.3,0.03,20.32,19.55,0.77,-0.28,-0.49,-328.0,0.60,0.07,0.05,0.03,0.04,0.74,332.0,-364.293,-1062.493,797.0,23.0,9.0,77.0,696.293,-730.493,62.889,269.111,91.889,410.889,463.889,-526.111,0.85,0.003,0.015,0.074,0.144,-92656917.0,-6593650.0,-28616261.0,-24814262.0,-59411238.0,0.067
17783,24.0,9.0,5.0,4.0,53998.0,549.43,0.74,0.86,0.08,-0.77,-1.22,0.169,0.236,0.584,0.587,0.762,551.04,548.37,2.67,-1.61,-1.06,45.37,27.73,13.62,12.76,-7.87,0.743,0.022,0.060,0.131,0.154,-1.02,0.07,-0.180,-0.446,0.0,-0.38,0.98,1.66,0.65,-0.080,-0.058,-0.279,0.618,0.305,-4.63,0.253,0.077,-0.081,-0.195,-1.068,0.089,-0.008,-0.063,-0.811,0.079,-0.107,-0.

In [2]:
current_directory = os.getcwd()
cleaned_data = pd.DataFrame()

#Import raw SPY data
relative_path = 'SPY_data_temp.csv'
csv_file = os.path.join(current_directory, relative_path)
SPY_raw = pd.read_csv(csv_file)

#Import raw SPY 5 min chart data
relative_path = 'SPY5_data_temp.csv'
csv_file = os.path.join(current_directory, relative_path)
SPY5_raw = pd.read_csv(csv_file)

#drop unneeded columns from SPY and SPY5 min data
SPY5.drop(columns='price', inplace=True)
SPY5.drop(columns='BBBasis', inplace=True)
SPY5.drop(columns='BBUpper', inplace=True)
SPY5.drop(columns='BBLower', inplace=True)
SPY5.drop(columns='MFI', inplace=True)
SPY5.drop(columns='ATR', inplace=True)
SPY5.drop(columns='Mom', inplace=True)
SPY.drop(columns='TTM', inplace=True)
SPY.drop(columns='color', inplace=True)

#Import raw TICK data
relative_path = 'TICK_data_temp.csv'
csv_file = os.path.join(current_directory, relative_path)
TICK_raw = pd.read_csv(csv_file)

#Import raw TRIN data
relative_path = 'TRIN_data_temp.csv'
csv_file = os.path.join(current_directory, relative_path)
TRIN_raw = pd.read_csv(csv_file)

#Import raw VIX data
relative_path = 'VIX_data_temp.csv'
csv_file = os.path.join(current_directory, relative_path)
VIX_raw = pd.read_csv(csv_file)

#Import raw ADD data
relative_path = 'ADD_data_temp.csv'
csv_file = os.path.join(current_directory, relative_path)
ADD_raw = pd.read_csv(csv_file)

#Import raw PCC data
relative_path = 'PCC_data_temp.csv'
csv_file = os.path.join(current_directory, relative_path)
PCC_raw = pd.read_csv(csv_file)

#Import raw VOLD data
relative_path = 'VOLD_data_temp.csv'
csv_file = os.path.join(current_directory, relative_path)
VOLD_raw = pd.read_csv(csv_file)


for year in range(SPY_raw.iloc[0]['year'], SPY_raw.iloc[-1]['year'] + 1):
    SPY_year = SPY_raw[SPY_raw['year'] == year]
    SPY5_year = SPY5_raw[SPY5_raw['year'] == year] 
    TICK_year = TICK_raw[TICK_raw['year'] == year] 
    TRIN_year = TRIN_raw[TRIN_raw['year'] == year] 
    VIX_year = VIX_raw[VIX_raw['year'] == year] 
    ADD_year = ADD_raw[ADD_raw['year'] == year] 
    PCC_year = PCC_raw[PCC_raw['year'] == year]
    VOLD_year = VOLD_raw[VOLD_raw['year'] == year] 
    for month in range(SPY_year.iloc[0]['month'], SPY_year.iloc[-1]['month'] + 1):
        print(month) #for monitering progress
        SPY_month = SPY_year[SPY_year['month'] == month]
        SPY5_month = SPY5_year[SPY5_year['month'] == month] 
        TICK_month = TICK_year[TICK_year['month'] == month] 
        TRIN_month = TRIN_year[TRIN_year['month'] == month] 
        VIX_month = VIX_year[VIX_year['month'] == month] 
        ADD_month = ADD_year[ADD_year['month'] == month] 
        PCC_month = PCC_year[PCC_year['month'] == month]
        VOLD_month = VOLD_year[VOLD_year['month'] == month]
        for day_num in range(SPY_month.iloc[0]['num'], SPY_month.iloc[-1]['num'] + 1):
            SPY = SPY_month[SPY_month['num'] == day_num]
            SPY5 = SPY5_month[SPY5_month['num'] == day_num] 
            TICK = TICK_month[TICK_month['num'] == day_num] 
            TRIN = TRIN_month[TRIN_month['num'] == day_num] 
            VIX = VIX_month[VIX_month['num'] == day_num] 
            ADD = ADD_month[ADD_month['num'] == day_num] 
            PCC = PCC_month[PCC_month['num'] == day_num]
            VOLD = VOLD_month[VOLD_month['num'] == day_num]

            #if markets were closed on the given day, jump to next day
            if SPY.empty:
                continue
            if (day_num == 29 and month == 11) or (day_num == 24 and month == 12):
                data = pd.DataFrame()
                data['time'] = range(30600, 43200)
            else:
                #initalizing blank dataframe to hold merged data from the day
                data = pd.DataFrame()
                data['time'] = range(30600, 54000)
            
            #convert the time column of each from hh:mm:ss, to int(seconds)
            SPY5['time'] = SPY5['time'] + ':00'
            SPY['time'] = SPY['time'].apply(time_to_seconds)
            SPY5['time'] = SPY5['time'].apply(time_to_seconds)
            TICK['time'] = TICK['time'].apply(time_to_seconds)
            TRIN['time'] = TRIN['time'].apply(time_to_seconds)
            VIX['time'] = VIX['time'].apply(time_to_seconds)
            ADD['time'] = ADD['time'].apply(time_to_seconds)
            PCC['time'] = PCC['time'].apply(time_to_seconds)
            VOLD['time'] = VOLD['time'].apply(time_to_seconds)

            # Apply the mapping to the 'Day' column using the map() method
            SPY['DayNumber'] = SPY['day'].map(day_to_number)
            SPY = SPY.drop('day', axis=1)
            SPY.rename(columns={'DayNumber': 'day'}, inplace=True)

            SPY5['DayNumber'] = SPY5['day'].map(day_to_number)
            SPY5 = SPY5.drop('day', axis=1)
            SPY5.rename(columns={'DayNumber': 'day'}, inplace=True)

            TICK['DayNumber'] = TICK['day'].map(day_to_number)
            TICK = TICK.drop('day', axis=1)
            TICK.rename(columns={'DayNumber': 'day'}, inplace=True)

            TRIN['DayNumber'] = TRIN['day'].map(day_to_number)
            TRIN = TRIN.drop('day', axis=1)
            TRIN.rename(columns={'DayNumber': 'day'}, inplace=True)

            VIX['DayNumber'] = VIX['day'].map(day_to_number)
            VIX = VIX.drop('day', axis=1)
            VIX.rename(columns={'DayNumber': 'day'}, inplace=True)

            ADD['DayNumber'] = ADD['day'].map(day_to_number)
            ADD = ADD.drop('day', axis=1)
            ADD.rename(columns={'DayNumber': 'day'}, inplace=True)

            PCC['DayNumber'] = PCC['day'].map(day_to_number)
            PCC = PCC.drop('day', axis=1)
            PCC.rename(columns={'DayNumber': 'day'}, inplace=True)

            VOLD['DayNumber'] = VOLD['day'].map(day_to_number)
            VOLD = VOLD.drop('day', axis=1)
            VOLD.rename(columns={'DayNumber': 'day'}, inplace=True)

            # Apply the mapping to the 'Day' column using the map() method
            SPY['MonthNumber'] = SPY['month'].map(month_to_number)
            SPY = SPY.drop('month', axis=1)
            SPY.rename(columns={'MonthNumber': 'month'}, inplace=True)

            SPY5['MonthNumber'] = SPY5['month'].map(month_to_number)
            SPY5 = SPY5.drop('month', axis=1)
            SPY5.rename(columns={'MonthNumber': 'month'}, inplace=True)

            TICK['MonthNumber'] = TICK['month'].map(month_to_number)
            TICK = TICK.drop('month', axis=1)
            TICK.rename(columns={'MonthNumber': 'month'}, inplace=True)

            TRIN['MonthNumber'] = TRIN['month'].map(month_to_number)
            TRIN = TRIN.drop('month', axis=1)
            TRIN.rename(columns={'MonthNumber': 'month'}, inplace=True)

            VIX['MonthNumber'] = VIX['month'].map(month_to_number)
            VIX = VIX.drop('month', axis=1)
            VIX.rename(columns={'MonthNumber': 'month'}, inplace=True)

            ADD['MonthNumber'] = ADD['month'].map(month_to_number)
            ADD = ADD.drop('month', axis=1)
            ADD.rename(columns={'MonthNumber': 'month'}, inplace=True)

            PCC['MonthNumber'] = PCC['month'].map(month_to_number)
            PCC = PCC.drop('month', axis=1)
            PCC.rename(columns={'MonthNumber': 'month'}, inplace=True)

            VOLD['MonthNumber'] = VOLD['month'].map(month_to_number)
            VOLD = VOLD.drop('month', axis=1)
            VOLD.rename(columns={'MonthNumber': 'month'}, inplace=True)

            #ensure no duplicate values exist by removing any duplicates from each dataset
            SPY = SPY.reset_index(drop=True)
            mask = (SPY['time'] == SPY['time'].shift(-1)) & (SPY['price'] == SPY['price'].shift(-1))
            SPY = SPY[~mask]
            SPY = SPY.reset_index(drop=True)

            SPY5 = SPY5.reset_index(drop=True)
            mask = (SPY5['time'] == SPY5['time'].shift(-1)) & (SPY5['price'] == SPY5['price'].shift(-1))
            SPY5 = SPY5[~mask]
            SPY5 = SPY5.reset_index(drop=True)

            TICK = TICK.reset_index(drop=True)
            mask = (TICK['time'] == TICK['time'].shift(-1)) & (TICK['TICK'] == TICK['TICK'].shift(-1))
            TICK = TICK[~mask]
            TICK = TICK.reset_index(drop=True)

            TRIN = TRIN.reset_index(drop=True)
            mask = (TRIN['time'] == TRIN['time'].shift(-1)) & (TRIN['TRIN'] == TRIN['TRIN'].shift(-1))
            TRIN = TRIN[~mask]
            TRIN = TRIN.reset_index(drop=True)

            VIX = VIX.reset_index(drop=True)
            mask = (VIX['time'] == VIX['time'].shift(-1)) & (VIX['VIX'] == VIX['VIX'].shift(-1))
            VIX = VIX[~mask]
            VIX = VIX.reset_index(drop=True)

            ADD = ADD.reset_index(drop=True)
            mask = (ADD['time'] == ADD['time'].shift(-1)) & (ADD['ADD'] == ADD['ADD'].shift(-1))
            ADD = ADD[~mask]
            ADD = ADD.reset_index(drop=True)

            PCC = PCC.reset_index(drop=True)
            mask = (PCC['time'] == PCC['time'].shift(-1)) & (PCC['PCC'] == PCC['PCC'].shift(-1))
            PCC = PCC[~mask]
            PCC = PCC.reset_index(drop=True)

            VOLD = VOLD.reset_index(drop=True)
            mask = (VOLD['time'] == VOLD['time'].shift(-1)) & (VOLD['VOLD'] == VOLD['VOLD'].shift(-1))
            VOLD = VOLD[~mask]
            VOLD = VOLD.reset_index(drop=True)

            #adding 'TRIN_lastclose' column based on the data from previous day's last TRIN value
            TRIN['TRIN_lastclose'] = last_TRIN

            #set the date based on the incoming data
            data['year'] = SPY.iloc[-1]['year']
            data['num'] = SPY.iloc[-1]['num']
            data['month'] = SPY.iloc[-1]['month']

            #merge all incoming data to new dataframe
            data = pd.merge(data, SPY, on=['year', 'month', 'num', 'time'], how='outer')
            data = pd.merge(data, SPY5, on=['year', 'month', 'num', 'day', 'time'], how='outer')            
            data = pd.merge(data, TICK, on=['year', 'month', 'num', 'day', 'time'], how='outer')
            data = pd.merge(data, ADD, on=['year', 'month', 'num', 'day', 'time'], how='outer')
            data = pd.merge(data, PCC, on=['year', 'month', 'num', 'day', 'time'], how='outer')
            data = pd.merge(data, TRIN, on=['year', 'month', 'num', 'day', 'time'], how='outer')
            data = pd.merge(data, VIX, on=['year', 'month', 'num', 'day', 'time'], how='outer')
            data = pd.merge(data, VOLD, on=['year', 'month', 'num', 'day', 'time'], how='outer')

            #sort the new data chronologically
            data_temp = data[(data['num'] == SPY.iloc[-1]['num']) & (data['month'] == SPY.iloc[-1]['month'])]
            sorted_df = data_temp.sort_values(by='time', ascending=True)
            df = sorted_df
            df = df.reset_index(drop=True) #clean up index values

            #remove any possible duplicate values
            mask = (df['time'] == df['time'].shift(-1)) & (df['price'] == df['price'].shift(-1))
            df = df[~mask]
            df = df.reset_index(drop=True) #clean up index values

            #append new data to the back of previous day's data to aquire recent values
            df = pd.concat([prev_day_data, df], ignore_index=False)

            #if NA values exist, fill in missing values with the most recent value available at that time since a no change state was detected
            df['TRIN'].fillna(method='ffill', inplace=True)
            df['TRIN_lastclose'].fillna(method='ffill', inplace=True)
            df['price'].fillna(method='ffill', inplace=True)
            df = df.drop(columns='BBBasis')
            df['BBUpper'].fillna(method='ffill', inplace=True)
            df['BBLower'].fillna(method='ffill', inplace=True)
            df['MFI'].fillna(method='ffill', inplace=True)
            df['ATR'].fillna(method='ffill', inplace=True)
            df['TTM'].fillna(method='ffill', inplace=True)
            df['color'].fillna(method='ffill', inplace=True)
            df['WaveASlow'].fillna(method='ffill', inplace=True)
            df['WaveAFast'].fillna(method='ffill', inplace=True)
            df['WaveBSlow'].fillna(method='ffill', inplace=True)
            df['WaveBFast'].fillna(method='ffill', inplace=True)
            df['WaveCMCAD'].fillna(method='ffill', inplace=True)
            df['WaveCFast'].fillna(method='ffill', inplace=True)
            df['Mom'].fillna(method='ffill', inplace=True)
            df['ADD'].fillna(method='ffill', inplace=True)
            df['TICK'].fillna(method='ffill', inplace=True)
            df['TICKBBUpper'].fillna(method='ffill', inplace=True)
            df['TICKBBLower'].fillna(method='ffill', inplace=True)
            df['TICKMA'].fillna(method='ffill', inplace=True)
            df['VOLD'].fillna(method='ffill', inplace=True)
            df['VIXBBUpper'].fillna(method='ffill', inplace=True)
            df['VIXBBLower'].fillna(method='ffill', inplace=True)
            df['VIX'].fillna(method='ffill', inplace=True)
            df['PCC'].fillna(method='ffill', inplace=True)
            df['TICK'].fillna(method='bfill', inplace=True)
            df['TICKBBUpper'].fillna(method='bfill', inplace=True)
            df['TICKBBLower'].fillna(method='bfill', inplace=True)
            df['TICKMA'].fillna(method='bfill', inplace=True)
            df['VOLD'].fillna(method='bfill', inplace=True)
            df['PCC'].fillna(method='bfill', inplace=True)
            df['ADD'].fillna(method='bfill', inplace=True)
            df['VIXBBUpper'].fillna(method='bfill', inplace=True)
            df['VIXBBLower'].fillna(method='bfill', inplace=True)
            df['VIX'].fillna(method='bfill', inplace=True)
            df['TRIN'].fillna(method='bfill', inplace=True)

            #price 5 minutes ago
            df['price_5_ago'] = df['price'].shift(300)
            df['change_5_ago'] = df['price'] - df['price_5_ago']

            #price 15 minutes ago
            df['price_15_ago'] = df['price'].shift(900)
            df['change_15_ago'] = df['price'] - df['price_15_ago']

            #price 30 minutes ago
            df['price_30_ago'] = df['price'].shift(1800)
            df['change_30_ago'] = df['price'] - df['price_30_ago']

            #price 1 hour ago
            df['price_60_ago'] = df['price'].shift(3600)
            df['change_60_ago'] = df['price'] - df['price_60_ago']

            #price 2 hours ago
            df['price_120_ago'] = df['price'].shift(7200)
            df['change_120_ago'] = df['price'] - df['price_120_ago']

            #drop excess columns used for calculation purposes
            df.drop(columns='price_5_ago', inplace=True)
            df.drop(columns='price_15_ago', inplace=True)
            df.drop(columns='price_30_ago', inplace=True)
            df.drop(columns='price_60_ago', inplace=True)
            df.drop(columns='price_120_ago', inplace=True)

            #standard deviation for the timeframes 5 min, 15 min, 30 min, 1 hr, 2hrs
            df['noise_5'] = df['price'].rolling(window=300).std()
            df['noise_15'] = df['price'].rolling(window=900).std()
            df['noise_30'] = df['price'].rolling(window=1800).std()
            df['noise_60'] = df['price'].rolling(window=3600).std()
            df['noise_120'] = df['price'].rolling(window=7200).std()

            #calculate the distances from important BB points
            df['BBdiff'] = df['BBUpper'] - df['BBLower']
            df['BBUpdiff'] = df['price'] - df['BBUpper']
            df['BBLowdiff'] = df['BBLower'] - df['price']

            #MFI 5 minutes ago
            df['MFI5'] = df['MFI'].shift(300)
            df['MFIchange_5'] = df['MFI'] - df['MFI5']
            df.drop(columns='MFI5', inplace=True)

            #MFI 15 minutes ago
            df['MFI15'] = df['MFI'].shift(900)
            df['MFIchange_15'] = df['MFI'] - df['MFI15']
            df.drop(columns='MFI15', inplace=True)

            #MFI 30 minutes ago
            df['MFI30'] = df['MFI'].shift(1800)
            df['MFIchange_30'] = df['MFI'] - df['MFI30']
            df.drop(columns='MFI30', inplace=True)

            #MFI 1 hour ago
            df['MFI60'] = df['MFI'].shift(3600)
            df['MFIchange_60'] = df['MFI'] - df['MFI60']
            df.drop(columns='MFI60', inplace=True)

            #ATR 5 minutes ago
            df['ATR5'] = df['ATR'].shift(300)
            df['ATRchange_5'] = df['ATR'] - df['ATR5']
            df.drop(columns='ATR5', inplace=True)

            #ATR 15 minutes ago
            df['ATR15'] = df['ATR'].shift(900)
            df['ATRchange_15'] = df['ATR'] - df['ATR15']
            df.drop(columns='ATR15', inplace=True)

            #ATR 30 minutes ago
            df['ATR30'] = df['ATR'].shift(1800)
            df['ATRchange_30'] = df['ATR'] - df['ATR30']
            df.drop(columns='ATR30', inplace=True)

            #ATR 60 minutes ago
            df['ATR60'] = df['ATR'].shift(3600)
            df['ATRchange_60'] = df['ATR'] - df['ATR60']
            df.drop(columns='ATR60', inplace=True)

            #Mom 5 minutes ago
            df['Mom5'] = df['Mom'].shift(300)
            df['Momchange_5'] = df['Mom'] - df['Mom5']
            df.drop(columns='Mom5', inplace=True)

            #Mom 15 minutes ago
            df['Mom15'] = df['Mom'].shift(900)
            df['Momchange_15'] = df['Mom'] - df['Mom15']
            df.drop(columns='Mom15', inplace=True)

            #Mom 30 minutes ago
            df['Mom30'] = df['Mom'].shift(1800)
            df['Momchange_30'] = df['Mom'] - df['Mom30']
            df.drop(columns='Mom30', inplace=True)

            #Mom 60 minutes ago
            df['Mom60'] = df['Mom'].shift(3600)
            df['Momchange_60'] = df['Mom'] - df['Mom60']
            df.drop(columns='Mom60', inplace=True)

            #TTM 5 minutes ago
            df['TTM5'] = df['TTM'].shift(300)
            df['TTMchange_5'] = df['TTM'] - df['TTM5']
            df.drop(columns='TTM5', inplace=True)

            #TTM 15 minutes ago
            df['TTM15'] = df['TTM'].shift(900)
            df['TTMchange_15'] = df['TTM'] - df['TTM15']
            df.drop(columns='TTM15', inplace=True)

            #TTM 30 minutes ago
            df['TTM30'] = df['TTM'].shift(1800)
            df['TTMchange_30'] = df['TTM'] - df['TTM30']
            df.drop(columns='TTM30', inplace=True)

            #WaveASlow 5 minutes ago
            df['WaveASlow5'] = df['WaveASlow'].shift(300)
            df['WaveASlowchange_5'] = df['WaveASlow'] - df['WaveASlow5']
            df.drop(columns='WaveASlow5', inplace=True)

            #WaveASlow 15 minutes ago
            df['WaveASlow15'] = df['WaveASlow'].shift(900)
            df['WaveASlowchange_15'] = df['WaveASlow'] - df['WaveASlow15']
            df.drop(columns='WaveASlow15', inplace=True)

            #WaveASlow 30 minutes ago
            df['WaveASlow30'] = df['WaveASlow'].shift(1800)
            df['WaveASlowchange_30'] = df['WaveASlow'] - df['WaveASlow30']
            df.drop(columns='WaveASlow30', inplace=True)

            #WaveASlow 1 hour ago
            df['WaveASlow60'] = df['WaveASlow'].shift(3600)
            df['WaveASlowchange_60'] = df['WaveASlow'] - df['WaveASlow60']
            df.drop(columns='WaveASlow60', inplace=True)

            #WaveAFast 5 minutes ago
            df['WaveAFast5'] = df['WaveAFast'].shift(300)
            df['WaveAFastchange_5'] = df['WaveAFast'] - df['WaveAFast5']
            df.drop(columns='WaveAFast5', inplace=True)

            #WaveAFast 15 minutes ago
            df['WaveAFast15'] = df['WaveAFast'].shift(900)
            df['WaveAFastchange_15'] = df['WaveAFast'] - df['WaveAFast15']
            df.drop(columns='WaveAFast15', inplace=True)

            #WaveAFast 30 minutes ago
            df['WaveAFast30'] = df['WaveAFast'].shift(1800)
            df['WaveAFastchange_30'] = df['WaveAFast'] - df['WaveAFast30']
            df.drop(columns='WaveAFast30', inplace=True)

            #WaveAFast 1 hour ago
            df['WaveAFast60'] = df['WaveAFast'].shift(3600)
            df['WaveAFastchange_60'] = df['WaveAFast'] - df['WaveAFast60']
            df.drop(columns='WaveAFast60', inplace=True)

            #WaveBSlow 5 minutes ago
            df['WaveBSlow5'] = df['WaveBSlow'].shift(300)
            df['WaveBSlowchange_5'] = df['WaveBSlow'] - df['WaveBSlow5']
            df.drop(columns='WaveBSlow5', inplace=True)

            #WaveBSlow 15 minutes ago
            df['WaveBSlow15'] = df['WaveBSlow'].shift(900)
            df['WaveBSlowchange_15'] = df['WaveBSlow'] - df['WaveBSlow15']
            df.drop(columns='WaveBSlow15', inplace=True)

            #WaveBSlow 30 minutes ago
            df['WaveBSlow30'] = df['WaveBSlow'].shift(1800)
            df['WaveBSlowchange_30'] = df['WaveBSlow'] - df['WaveBSlow30']
            df.drop(columns='WaveBSlow30', inplace=True)

            #WaveBSlow 1 hour ago
            df['WaveBSlow60'] = df['WaveBSlow'].shift(3600)
            df['WaveBSlowchange_60'] = df['WaveBSlow'] - df['WaveBSlow60']
            df.drop(columns='WaveBSlow60', inplace=True)

            #WaveBFast 5 minutes ago
            df['WaveBFast5'] = df['WaveBFast'].shift(300)
            df['WaveBFastchange_5'] = df['WaveBFast'] - df['WaveBFast5']
            df.drop(columns='WaveBFast5', inplace=True)

            #WaveBFast 15 minutes ago
            df['WaveBFast15'] = df['WaveBFast'].shift(900)
            df['WaveBFastchange_15'] = df['WaveBFast'] - df['WaveBFast15']
            df.drop(columns='WaveBFast15', inplace=True)

            #WaveBFast 30 minutes ago
            df['WaveBFast30'] = df['WaveBFast'].shift(1800)
            df['WaveBFastchange_30'] = df['WaveBFast'] - df['WaveBFast30']
            df.drop(columns='WaveBFast30', inplace=True)

            #WaveBFast 1 hour ago
            df['WaveBFast60'] = df['WaveBFast'].shift(3600)
            df['WaveBFastchange_60'] = df['WaveBFast'] - df['WaveBFast60']
            df.drop(columns='WaveBFast60', inplace=True)

            #WaveCMCAD 5 minutes ago
            df['WaveCMCAD5'] = df['WaveCMCAD'].shift(300)
            df['WaveCMCADchange_5'] = df['WaveCMCAD'] - df['WaveCMCAD5']
            df.drop(columns='WaveCMCAD5', inplace=True)

            #WaveCMCAD 15 minutes ago
            df['WaveCMCAD15'] = df['WaveCMCAD'].shift(900)
            df['WaveCMCADchange_15'] = df['WaveCMCAD'] - df['WaveCMCAD15']
            df.drop(columns='WaveCMCAD15', inplace=True)

            #WaveCMCAD 30 minutes ago
            df['WaveCMCAD30'] = df['WaveCMCAD'].shift(1800)
            df['WaveCMCADchange_30'] = df['WaveCMCAD'] - df['WaveCMCAD30']
            df.drop(columns='WaveCMCAD30', inplace=True)

            #WaveCMCAD 1 hour ago
            df['WaveCMCAD60'] = df['WaveCMCAD'].shift(3600)
            df['WaveCMCADchange_60'] = df['WaveCMCAD'] - df['WaveCMCAD60']
            df.drop(columns='WaveCMCAD60', inplace=True)

            #WaveCFast 5 minutes ago
            df['WaveCFast5'] = df['WaveCFast'].shift(300)
            df['WaveCFastchange_5'] = df['WaveCFast'] - df['WaveCFast5']
            df.drop(columns='WaveCFast5', inplace=True)

            #WaveCFast 15 minutes ago
            df['WaveCFast15'] = df['WaveCFast'].shift(900)
            df['WaveCFastchange_15'] = df['WaveCFast'] - df['WaveCFast15']
            df.drop(columns='WaveCFast15', inplace=True)

            #WaveCFast 30 minutes ago
            df['WaveCFast30'] = df['WaveCFast'].shift(1800)
            df['WaveCFastchange_30'] = df['WaveCFast'] - df['WaveCFast30']
            df.drop(columns='WaveCFast30', inplace=True)

            #WaveCFast 1 hour ago
            df['WaveCFast60'] = df['WaveCFast'].shift(3600)
            df['WaveCFastchange_60'] = df['WaveCFast'] - df['WaveCFast60']
            df.drop(columns='WaveCFast60', inplace=True)

            #VIX 5 minutes ago
            df['VIX5'] = df['VIX'].shift(300)
            df['VIXchange_5'] = df['VIX'] - df['VIX5']
            df.drop(columns='VIX5', inplace=True)

            #VIX 15 minutes ago
            df['VIX15'] = df['VIX'].shift(900)
            df['VIXchange_15'] = df['VIX'] - df['VIX15']
            df.drop(columns='VIX15', inplace=True)

            #VIX 30 minutes ago
            df['VIX30'] = df['VIX'].shift(1800)
            df['VIXchange_30'] = df['VIX'] - df['VIX30']
            df.drop(columns='VIX30', inplace=True)

            #VIX 1 hour ago
            df['VIX60'] = df['VIX'].shift(3600)
            df['VIXchange_60'] = df['VIX'] - df['VIX60']
            df.drop(columns='VIX60', inplace=True)

            #VIX 2 hours ago
            df['VIX120'] = df['VIX'].shift(7200)
            df['VIXchange_120'] = df['VIX'] - df['VIX120']
            df.drop(columns='VIX120', inplace=True)

            #calculate the distances from important VIXBB points
            df['VIXBBdiff'] = df['VIXBBUpper'] - df['VIXBBLower']
            df['VIXBBUpdiff'] = df['VIX'] - df['VIXBBUpper']
            df['VIXBBLowdiff'] = df['VIXBBLower'] - df['VIX']

            #TRIN 5 minutes ago
            df['TRIN5'] = df['TRIN'].shift(300)
            df['TRINchange_5'] = df['TRIN'] - df['TRIN5']
            df.drop(columns='TRIN5', inplace=True)

            #TRIN 15 minutes ago
            df['TRIN15'] = df['TRIN'].shift(900)
            df['TRINchange_15'] = df['TRIN'] - df['TRIN15']
            df.drop(columns='TRIN15', inplace=True)

            #TRIN 30 minutes ago
            df['TRIN30'] = df['TRIN'].shift(1800)
            df['TRINchange_30'] = df['TRIN'] - df['TRIN30']
            df.drop(columns='TRIN30', inplace=True)

            #TRIN 1 hour ago
            df['TRIN60'] = df['TRIN'].shift(3600)
            df['TRINchange_60'] = df['TRIN'] - df['TRIN60']
            df.drop(columns='TRIN60', inplace=True)

            #TICK 5 minutes ago
            df['TICK5'] = df['TICK'].shift(300)
            df['TICKchange_5'] = df['TICK'] - df['TICK5']
            df.drop(columns='TICK5', inplace=True)

            #TICK 15 minutes ago
            df['TICK15'] = df['TICK'].shift(900)
            df['TICKchange_15'] = df['TICK'] - df['TICK15']
            df.drop(columns='TICK15', inplace=True)

            #TICK 30 minutes ago
            df['TICK30'] = df['TICK'].shift(1800)
            df['TICKchange_30'] = df['TICK'] - df['TICK30']
            df.drop(columns='TICK30', inplace=True)

            #TICK 1 hour ago
            df['TICK60'] = df['TICK'].shift(3600)
            df['TICKchange_60'] = df['TICK'] - df['TICK60']
            df.drop(columns='TICK60', inplace=True)

            #calculate the distances from important TICKBB points
            df['TICKBBUpdiff'] = df['TICK'] - df['TICKBBUpper']
            df['TICKBBLowdiff'] = df['TICKBBLower'] - df['TICK']
            df['TICKMADiff'] = df['TICK'] - df['TICKMA']

            #TICKMA 5 minutes ago
            df['TICKMA5'] = df['TICKMA'].shift(300)
            df['TICKMAchange_5'] = df['TICKMA'] - df['TICKMA5']
            df.drop(columns='TICKMA5', inplace=True)
    
            #TICKMA 15 minutes ago
            df['TICKMA15'] = df['TICKMA'].shift(900)
            df['TICKMAchange_15'] = df['TICKMA'] - df['TICKMA15']
            df.drop(columns='TICKMA15', inplace=True)

            #TICKMA 30 minutes ago
            df['TICKMA30'] = df['TICKMA'].shift(1800)
            df['TICKMAchange_30'] = df['TICKMA'] - df['TICKMA30']
            df.drop(columns='TICKMA30', inplace=True)

            #TICKMA 1 hour ago
            df['TICKMA60'] = df['TICKMA'].shift(3600)
            df['TICKMAchange_60'] = df['TICKMA'] - df['TICKMA60']
            df.drop(columns='TICKMA60', inplace=True)

            #PCC 5 minutes ago
            df['PCC5'] = df['PCC'].shift(300)
            df['PCCchange_5'] = df['PCC'] - df['PCC5']
            df.drop(columns='PCC5', inplace=True)

            #PCC 15 minutes ago
            df['PCC15'] = df['PCC'].shift(900)
            df['PCCchange_15'] = df['PCC'] - df['PCC15']
            df.drop(columns='PCC15', inplace=True)

            #PCC 30 minutes ago
            df['PCC30'] = df['PCC'].shift(1800)
            df['PCCchange_30'] = df['PCC'] - df['PCC30']
            df.drop(columns='PCC30', inplace=True)

            #PCC 1 hour ago
            df['PCC60'] = df['PCC'].shift(3600)
            df['PCCchange_60'] = df['PCC'] - df['PCC60']
            df.drop(columns='PCC60', inplace=True)

            #VOLD 5 minutes ago
            df['VOLD5'] = df['VOLD'].shift(300)
            df['VOLDchange_5'] = df['VOLD'] - df['VOLD5']
            df.drop(columns='VOLD5', inplace=True)

            #VOLD 15 minutes ago
            df['VOLD15'] = df['VOLD'].shift(900)
            df['VOLDchange_15'] = df['VOLD'] - df['VOLD15']
            df.drop(columns='VOLD15', inplace=True)

            #VOLD 30 minutes ago
            df['VOLD30'] = df['VOLD'].shift(1800)
            df['VOLDchange_30'] = df['VOLD'] - df['VOLD30']
            df.drop(columns='VOLD30', inplace=True)

            #VOLD 1 hour ago
            df['VOLD60'] = df['VOLD'].shift(3600)
            df['VOLDchange_60'] = df['VOLD'] - df['VOLD60']
            df.drop(columns='VOLD60', inplace=True)

            #remove prior day's data leaving only new day's data
            df = df[df['num'] == df.iloc[-1]['num']]
            cleaned_data = pd.concat([cleaned_data, df], ignore_index=False)
            prev_day_data = df
#double check for correctness
cleaned_data

C:\Users\Derek\AppData\Local\Temp\ipykernel_15668\1354467622.py:220: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['TRIN'].fillna(method='ffill', inplace=True)
C:\Users\Derek\AppData\Local\Temp\ipykernel_15668\1354467622.py:220: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['TRIN'].fillna(method='ffill', inplace=True)
C:\Users\Derek\AppData\Local\Temp\ipykernel_15668\1354467622.py:221: FutureWarning: A value is trying to be set on a 

,year,month,num,day,time,price,change_5_ago,change_15_ago,change_30_ago,change_60_ago,change_120_ago,noise_5,noise_15,noise_30,noise_60,noise_120,BBUpper,BBLower,BBdiff,BBUpdiff,BBLowdiff,MFI,MFIchange_5,MFIchange_15,MFIchange_30,MFIchange_60,ATR,ATRchange_5,ATRchange_15,ATRchange_30,ATRchange_60,TTM,TTMchange_5,TTMchange_15,TTMchange_30,color,Mom,Momchange_5,Momchange_15,Momchange_30,Momchange_60,WaveASlow,WaveAFast,WaveBSlow,WaveBFast,WaveCMCAD,WaveCFast,WaveASlowchange_5,WaveASlowchange_15,WaveASlowchange_30,WaveASlowchange_60,WaveAFastchange_5,WaveAFastchange_15,WaveAFastchange_30,WaveAFastchange_60,WaveBSlowchange_5,WaveBSlowchange_15,WaveBSlowchange_30,WaveBSlowchange_60,WaveBFastchange_5,WaveBFastchange_15,WaveBFastchange_30,WaveBFastchange_60,WaveCMCADchange_5,WaveCMCADchange_15,WaveCMCADchange_30,WaveCMCADchange_60,WaveCFastchange_5,WaveCFastchange_15,WaveCFastchange_30,WaveCFastchange_60,VIX,VIXchange_5,VIXchange_15,VIXchange_30,VIXchange_60,VIXchange_120,VIXBBUpper,VIXBBLower,VIXBBdiff,VIXBBUpdiff,VIXBBLowdiff,ADD,TRIN,TRINchange_5,TRINchange_15,TRINchange_30,TRINchange_60,TRIN_lastclose,TICK,TICKBBUpdiff,TICKBBLowdiff,TICKchange_5,TICKchange_15,TICKchange_30,TICKchange_60,TICKBBUpper,TICKBBLower,TICKMA,TICKMADiff,TICKMAchange_5,TICKMAchange_15,TICKMAchange_30,TICKMAchange_60,PCC,PCCchange_5,PCCchange_15,PCCchange_30,PCCchange_60,VOLD,VOLDchange_5,VOLDchange_15,VOLDchange_30,VOLDchange_60,change_5,BBBasis
0,24.0,8.0,26.0,1.0,30600.0,563.18,1.16,1.80,1.19,2.20,2.63,0.158391,0.464149,0.372588,0.557732,0.559635,562.46,560.20,2.26,0.72,-2.98,68.13,17.29,9.66,-3.62,1.43,0.6235,0.0365,0.0965,0.1155,0.1505,0.8893,0.4633,0.5463,0.4783,0.0,1.21,-0.34,0.860,0.03,0.875,0.7880,0.5135,1.5900,1.2400,4.49,1.12,0.3740,0.3790,0.2590,0.4830,0.3085,0.3055,0.1665,0.3385,0.4700,0.5100,0.4500,0.7900,0.4240,0.4410,0.3410,0.6060,0.55,0.63,0.63,1.07,0.527,0.60,0.5890,1.016,15.72,-0.01,-0.25,-0.11,-0.35,-0.49,16.33,15.65,0.68,-0.61,-0.07,2161.0,0.41,-0.59,-0.59,-0.58,-0.67,0.98,384.0,-352.625,-833.825,130.0,571.0,-103.0,459.0,736.625,-449.825,283.000,101.000,42.556,91.222,92.444,315.333,1.082,-0.006,-0.019,-0.039,-0.097,382359279.0,65923855.0,84845010.0,94739549.0,126520373.0,NaN,561.33
1,24.0,8.0,26.0,1.0,30601.0,563.35,1.36,1.97,1.35,2.36,2.80,0.168802,0.466882,0.374356,0.558563,0.560227,562.46,560.20,2.26,0.89,-3.15,68.13,17.27,9.66,-3.63,1.49,0.6235,0.0365,0.0965,0.1155,0.1505,0.8893,0.4693,0.5463,0.4763,0.0,1.21,-0.31,0.860,0.02,0.860,0.7880,0.5135,1.5900,1.2400,4.49,1.12,0.3790,0.3790,0.2580,0.4830,0.3125,0.3055,0.1655,0.3385,0.4800,0.5100,0.4500,0.7870,0.4300,0.4410,0.3400,0.6040,0.55,0.63,0.63,1.06,0.533,0.60,0.5870,1.012,15.72,-0.01,-0.25,-0.11,-0.35,-0.49,16.33,15.65,0.68,-0.61,-0.07,2161.0,0.41,-0.59,-0.59,-0.58,-0.67,0.98,1114.0,403.577,-1571.923,876.0,1299.0,629.0,1191.0,710.423,-457.923,244.889,869.111,6.222,52.889,54.556,277.445,1.082,-0.006,-0.019,-0.039,-0.097,382359279.0,65894309.0,84836434.0,94717412.0,127915060.0,NaN,561.33
2,24.0,8.0,26.0,1.0,30602.0,563.34,1.39,1.96,1.34,2.35,2.79,0.178047,0.469553,0.376091,0.559383,0.560812,562.46,560.20,2.26,0.88,-3.14,68.13,17.26,9.66,-3.63,1.53,0.6235,0.0365,0.0965,0.1155,0.1505,0.8893,0.4813,0.5463,0.4763,0.0,1.21,-0.24,0.860,0.02,0.840,0.7880,0.5135,1.5900,1.2400,4.49,1.12,0.3920,0.3790,0.2580,0.4800,0.3235,0.3055,0.1655,0.3355,0.4900,0.5100,0.4500,0.7830,0.4430,0.4410,0.3400,0.6000,0.57,0.63,0.63,1.06,0.548,0.60,0.5870,1.008,15.72,-0.01,-0.25,-0.11,-0.35,-0.49,16.33,15.65,0.68,-0.61,-0.07,2161.0,0.41,-0.59,-0.59,-0.58,-0.67,0.98,1114.0,403.577,-1571.923,972.0,1301.0,631.0,1189.0,710.423,-457.923,244.889,869.111,16.889,53.111,54.778,277.222,1.082,-0.006,-0.019,-0.037,-0.097,382359279.0,66878172.0,84820537.0,94706308.0,127912644.0,NaN,561.33
3,24.0,8.0,26.0,1.0,30603.0,563.30,1.44,1.92,1.28,2.29,2.75,0.185341,0.472054,0.377720,0.560168,0.561374,562.46,560.20,2.26,0.84,-3.10,68.13,17.24,9.66,-3.64,1.53,0.6235,0.0365,0.0965,0.1155,0.1505,0.8893,0.4813,0.5463,0.4733,0.0,1.21,-0.24,0.8

In [3]:
#defining features for ML model 
X = cleaned_data[['time', 'MFI', 'ATR', 'TTM', 'color', 'Mom', 'WaveASlow', 'WaveAFast', 'WaveBSlow', 'WaveBFast',
                    'WaveCMCAD', 'WaveCFast', 'day', 'TICKBBUpper', 'TICKBBLower', 'TICKMA', 'TICK', 'ADD', 'PCC',
                    'TRIN', 'TRIN_lastclose', 'VIX', 'VOLD', 'change_5_ago', 'change_15_ago', 'change_30_ago', 'change_60_ago',
                    'change_120_ago', 'BBdiff', 'BBUpdiff', 'BBLowdiff', 'MFIchange_5', 'MFIchange_15', 'MFIchange_30', 'MFIchange_60',
                    'ATRchange_5', 'ATRchange_15', 'ATRchange_30', 'ATRchange_60', 'Momchange_5', 'Momchange_15', 'Momchange_30', 
                    'Momchange_60', 'TTMchange_5', 'TTMchange_15', 'TTMchange_30', 'WaveASlowchange_5', 'WaveASlowchange_15',
                    'WaveASlowchange_30', 'WaveASlowchange_60', 'WaveAFastchange_5', 'WaveAFastchange_15', 'WaveAFastchange_30', 
                    'WaveAFastchange_60', 'WaveBSlowchange_5', 'WaveBSlowchange_15', 'WaveBSlowchange_30', 'WaveBSlowchange_60',
                    'WaveBFastchange_5', 'WaveBFastchange_15', 'WaveBFastchange_30', 'WaveBFastchange_60', 'WaveCMCADchange_5',
                    'WaveCMCADchange_15', 'WaveCMCADchange_30', 'WaveCMCADchange_60', 'WaveCFastchange_5', 'WaveCFastchange_15',
                    'WaveCFastchange_30', 'WaveCFastchange_60', 'VIXchange_5', 'VIXchange_15', 'VIXchange_30', 'VIXchange_60',
                    'VIXchange_120', 'VIXBBdiff', 'VIXBBUpdiff', 'VIXBBLowdiff', 'TRINchange_5','TRINchange_15', 'TRINchange_30', 
                    'TRINchange_60', 'TICKchange_5', 'TICKchange_15', 'TICKchange_30', 'TICKchange_60', 'TICKBBUpdiff', 'TICKBBLowdiff',
                    'TICKMAchange_5', 'TICKMAchange_15', 'TICKMAchange_30', 'TICKMAchange_60', 'TICKMADiff','PCCchange_5', 'PCCchange_15', 
                    'PCCchange_30', 'PCCchange_60','VOLDchange_5', 'VOLDchange_15', 'VOLDchange_30', 'VOLDchange_60']]

#import ML model
change5_model = joblib.load('SPY5Models/change_5_model_8-24.joblib')

#predict change that will occur in 5 minutes
cleaned_data['change_5'] = change5_model.predict(X)

#verify outputs
cleaned_data

,year,month,num,day,time,price,change_5_ago,change_15_ago,change_30_ago,change_60_ago,change_120_ago,noise_5,noise_15,noise_30,noise_60,noise_120,BBUpper,BBLower,BBdiff,BBUpdiff,BBLowdiff,MFI,MFIchange_5,MFIchange_15,MFIchange_30,MFIchange_60,ATR,ATRchange_5,ATRchange_15,ATRchange_30,ATRchange_60,TTM,TTMchange_5,TTMchange_15,TTMchange_30,color,Mom,Momchange_5,Momchange_15,Momchange_30,Momchange_60,WaveASlow,WaveAFast,WaveBSlow,WaveBFast,WaveCMCAD,WaveCFast,WaveASlowchange_5,WaveASlowchange_15,WaveASlowchange_30,WaveASlowchange_60,WaveAFastchange_5,WaveAFastchange_15,WaveAFastchange_30,WaveAFastchange_60,WaveBSlowchange_5,WaveBSlowchange_15,WaveBSlowchange_30,WaveBSlowchange_60,WaveBFastchange_5,WaveBFastchange_15,WaveBFastchange_30,WaveBFastchange_60,WaveCMCADchange_5,WaveCMCADchange_15,WaveCMCADchange_30,WaveCMCADchange_60,WaveCFastchange_5,WaveCFastchange_15,WaveCFastchange_30,WaveCFastchange_60,VIX,VIXchange_5,VIXchange_15,VIXchange_30,VIXchange_60,VIXchange_120,VIXBBUpper,VIXBBLower,VIXBBdiff,VIXBBUpdiff,VIXBBLowdiff,ADD,TRIN,TRINchange_5,TRINchange_15,TRINchange_30,TRINchange_60,TRIN_lastclose,TICK,TICKBBUpdiff,TICKBBLowdiff,TICKchange_5,TICKchange_15,TICKchange_30,TICKchange_60,TICKBBUpper,TICKBBLower,TICKMA,TICKMADiff,TICKMAchange_5,TICKMAchange_15,TICKMAchange_30,TICKMAchange_60,PCC,PCCchange_5,PCCchange_15,PCCchange_30,PCCchange_60,VOLD,VOLDchange_5,VOLDchange_15,VOLDchange_30,VOLDchange_60,change_5,BBBasis
0,24.0,8.0,26.0,1.0,30600.0,563.18,1.16,1.80,1.19,2.20,2.63,0.158391,0.464149,0.372588,0.557732,0.559635,562.46,560.20,2.26,0.72,-2.98,68.13,17.29,9.66,-3.62,1.43,0.6235,0.0365,0.0965,0.1155,0.1505,0.8893,0.4633,0.5463,0.4783,0.0,1.21,-0.34,0.860,0.03,0.875,0.7880,0.5135,1.5900,1.2400,4.49,1.12,0.3740,0.3790,0.2590,0.4830,0.3085,0.3055,0.1665,0.3385,0.4700,0.5100,0.4500,0.7900,0.4240,0.4410,0.3410,0.6060,0.55,0.63,0.63,1.07,0.527,0.60,0.5890,1.016,15.72,-0.01,-0.25,-0.11,-0.35,-0.49,16.33,15.65,0.68,-0.61,-0.07,2161.0,0.41,-0.59,-0.59,-0.58,-0.67,0.98,384.0,-352.625,-833.825,130.0,571.0,-103.0,459.0,736.625,-449.825,283.000,101.000,42.556,91.222,92.444,315.333,1.082,-0.006,-0.019,-0.039,-0.097,382359279.0,65923855.0,84845010.0,94739549.0,126520373.0,0.146796,561.33
1,24.0,8.0,26.0,1.0,30601.0,563.35,1.36,1.97,1.35,2.36,2.80,0.168802,0.466882,0.374356,0.558563,0.560227,562.46,560.20,2.26,0.89,-3.15,68.13,17.27,9.66,-3.63,1.49,0.6235,0.0365,0.0965,0.1155,0.1505,0.8893,0.4693,0.5463,0.4763,0.0,1.21,-0.31,0.860,0.02,0.860,0.7880,0.5135,1.5900,1.2400,4.49,1.12,0.3790,0.3790,0.2580,0.4830,0.3125,0.3055,0.1655,0.3385,0.4800,0.5100,0.4500,0.7870,0.4300,0.4410,0.3400,0.6040,0.55,0.63,0.63,1.06,0.533,0.60,0.5870,1.012,15.72,-0.01,-0.25,-0.11,-0.35,-0.49,16.33,15.65,0.68,-0.61,-0.07,2161.0,0.41,-0.59,-0.59,-0.58,-0.67,0.98,1114.0,403.577,-1571.923,876.0,1299.0,629.0,1191.0,710.423,-457.923,244.889,869.111,6.222,52.889,54.556,277.445,1.082,-0.006,-0.019,-0.039,-0.097,382359279.0,65894309.0,84836434.0,94717412.0,127915060.0,0.171119,561.33
2,24.0,8.0,26.0,1.0,30602.0,563.34,1.39,1.96,1.34,2.35,2.79,0.178047,0.469553,0.376091,0.559383,0.560812,562.46,560.20,2.26,0.88,-3.14,68.13,17.26,9.66,-3.63,1.53,0.6235,0.0365,0.0965,0.1155,0.1505,0.8893,0.4813,0.5463,0.4763,0.0,1.21,-0.24,0.860,0.02,0.840,0.7880,0.5135,1.5900,1.2400,4.49,1.12,0.3920,0.3790,0.2580,0.4800,0.3235,0.3055,0.1655,0.3355,0.4900,0.5100,0.4500,0.7830,0.4430,0.4410,0.3400,0.6000,0.57,0.63,0.63,1.06,0.548,0.60,0.5870,1.008,15.72,-0.01,-0.25,-0.11,-0.35,-0.49,16.33,15.65,0.68,-0.61,-0.07,2161.0,0.41,-0.59,-0.59,-0.58,-0.67,0.98,1114.0,403.577,-1571.923,972.0,1301.0,631.0,1189.0,710.423,-457.923,244.889,869.111,16.889,53.111,54.778,277.222,1.082,-0.006,-0.019,-0.037,-0.097,382359279.0,66878172.0,84820537.0,94706308.0,127912644.0,0.199808,561.33
3,24.0,8.0,26.0,1.0,30603.0,563.30,1.44,1.92,1.28,2.29,2.75,0.185341,0.472054,0.377720,0.560168,0.561374,562.46,560.20,2.26,0.84,-3.10,68.13,17.24,9.66,-3.64,1.53,0.6235,0.0365,0.0965,0.1155,0.1505,0.8893,0.4813,0.5463,0.4733,0.0

In [6]:
#round all values to three decimal places
cleaned_data = cleaned_data.round(3)

#package all data into csv files based on date.
for year in range(cleaned_data.iloc[0]['year'], cleaned_data.iloc[-1]['year'] + 1):
    cleaned_data_year = cleaned_data[cleaned_data['year'] == year]
    for month in range(cleaned_data_year.iloc[0]['month'], cleaned_data.iloc[-1]['month'] + 1):
        cleaned_data_month = cleaned_data_year[cleaned_data_year['month'] == month]
        for day_num in range(cleaned_data_month.iloc[0]['num'], cleaned_data.iloc[-1]['num'] + 1):
            cleaned_data_day = cleaned_data_month[cleaned_data_month['num'] == day_num]
            if !cleaned_data_day.empty:
                file_path = f"dailydata/Sec_{month}-{day_num}-{year}.csv"
                cleaned_data_day.to_csv(file_path, index=False)
